In [1]:
import os
import time
import pickle
import networkx as nx
import numpy as np
from dtaidistance import dtw
import torch

In [2]:
def load_data(load_dir):

    merged_graphs = None
    
    """ 加载 Merged_Gaussian_DTW_graphs.pkl """
    dtw_graphs_path = os.path.join(load_dir, "Merged_Gaussian_DTW_graphs.pkl")

    with open(dtw_graphs_path, 'rb') as f:
        merged_graphs = pickle.load(f)

    print(f"✅ Successfully loaded {len(merged_graphs)} DTW graphs.")

    return merged_graphs

In [3]:
load_dir = r"/root/autodl-tmp/data/Antwerp_2019_march"
merged_graphs = load_data(load_dir)

✅ Successfully loaded 5524 DTW graphs.


In [4]:
merged_graphs_first = merged_graphs

In [5]:
def load_variable(filepath):
    """
    从文件中加载变量。
    """
    with open(filepath, 'rb') as f:
        data = pickle.load(f)
    return data

def readdata(filepath):
    """
    读取数据文件并转换为 NumPy 数组。
        """
    data = load_variable(filepath)
    data = np.array(data)
    print(f"Loaded data shape: {data.shape}")

    return data

In [6]:
march_half_filepath = r"/root/autodl-tmp/data/Antwerp_2020_april/Antwerp_2020april_last_half_month_samples_30min_timesteps.txt"
march_half = readdata(march_half_filepath)

Loaded data shape: (720, 495, 436)


In [7]:
class TrafficDataProcessor:
    def __init__(self, data, G_flat):
        self.data = data
        self.G_flat = G_flat
        self.valid_nodes = sorted(list(G_flat.nodes()))  # 经过扁平化的有效节点
        self.filtered_data = None

    def filter_and_flatten(self):
        """
        过滤数据，仅保留 G_flat 中的节点，转换形状 (time_steps, num_valid_nodes)。
        """
        time_steps = self.data.shape[0]

        # 将数据展平成 (time_steps, num_rows*num_cols)
        data_flat = self.data.reshape(time_steps, -1)
        
        # 仅保留 G_flat 中的有效节点
        self.filtered_data = data_flat[:, self.valid_nodes]
        
        print(f"Shape after filtered and flattened: {self.filtered_data.shape} (time_steps, num_valid_nodes)")
        return self.filtered_data, self.valid_nodes

    def save_sliding_window_with_dtw(self, save_dir, history_size=6, forecast_size=6):
        """
        生成滑动窗口数据，并计算DTW，存入G_flat的边权重中，并记录每个窗口的计算时间。
        """
        time_steps, num_valid_nodes = self.filtered_data.shape
        num_samples = time_steps - history_size - forecast_size + 1

        os.makedirs(save_dir, exist_ok=True)
        
        X_all = []
        Y_all = []
        dtw_graphs = []
        # time_records = []

        for i in range(num_samples):
            start_time = time.time()  # 记录起始时间

            # 取 [i, i+history_size) 这段做X
            hist_data = self.filtered_data[i : i + history_size, :]
            # 取 [i+history_size, i+history_size+forecast_size) 这段做Y
            fut_data = self.filtered_data[i + history_size : i + history_size + forecast_size, :]

            # 复制当前 G_flat 以存储DTW权重
            G_dtw = self.G_flat.copy()

            # 遍历图的所有边，计算DTW
            for u, v in G_dtw.edges():
                # print(f"Sequence 1: {hist_data[:, self.valid_nodes.index(u)].astype(np.float32)}")
                # print(f"Sequence 2: {hist_data[:, self.valid_nodes.index(v)].astype(np.float32)}")
                dtw_distance = dtw.distance(hist_data[:, self.valid_nodes.index(u)].astype(np.float32),
                                            hist_data[:, self.valid_nodes.index(v)].astype(np.float32))
                G_dtw[u][v]['DTW_weight'] = dtw_distance

            # node = 68  # 假设你想找节点 68 的边权重

            # # 遍历该节点的所有邻居，打印边的 DTW 权重
            # for neighbor, data in G_dtw[node].items():
            #     print(f"Edge {node} - {neighbor}, DTW Weight: {data['weight']}")

            # node = 505  # 假设你想找节点 68 的边权重

            # # 遍历该节点的所有邻居，打印边的 DTW 权重
            # for neighbor, data in G_dtw[node].items():
            #     print(f"Edge {node} - {neighbor}, DTW Weight: {data['weight']}")

            X_all.append(hist_data)
            Y_all.append(fut_data)
            dtw_graphs.append(G_dtw)

            end_time = time.time()  # 记录结束时间
            elapsed_time = end_time - start_time
            # time_records.append(elapsed_time)
            print(f"Sliding window {i+1}/{num_samples} is generated，cost: {elapsed_time:.4f} seconds")

        # 转换为 NumPy 数组
        X_all = np.array(X_all, dtype=np.float32)
        print(f"shape of X {X_all.shape}")
        Y_all = np.array(Y_all, dtype=np.float32)
        print(f"shape of Y {Y_all.shape}")
        
        # 保存数据
        np.save(os.path.join(save_dir, "X.npy"), X_all)
        np.save(os.path.join(save_dir, "Y.npy"), Y_all)
        np.save(os.path.join(save_dir, "time_records.npy"), np.array(time_records))
        
        # 保存 DTW 加权图对象（使用 pickle）
        with open(os.path.join(save_dir, "DTW_graphs.pkl"), 'wb') as f:
            pickle.dump(dtw_graphs, f)
        
        print(f"Data has been saved to '{save_dir}', shape: X {X_all.shape}, Y {Y_all.shape}.")
        print(f"Saved {len(dtw_graphs)} graphs with DTW weights.")
        # print(f"平均每个滑动窗口计算耗时: {np.mean(time_records):.4f} 秒")
        print('----------------------------------------------------------------------------------------------------------')

        return dtw_graphs

In [ ]:
# 使用示例   
if __name__ == "__main__":

    save_dir = r"/root/autodl-tmp/data"
    history_size = 6
    forecast_size = 6
    
    processor = TrafficDataProcessor(march_half, merged_graphs_first)
    filtered_data, valid_nodes = processor.filter_and_flatten()
    dtw_graphs = processor.save_sliding_window_with_dtw(save_dir, history_size=history_size, forecast_size=forecast_size)

Shape after filtered and flattened: (720, 5524) (time_steps, num_valid_nodes)
Sliding window 1/709 is generated，cost: 8.5256 seconds
Sliding window 2/709 is generated，cost: 8.4772 seconds
Sliding window 3/709 is generated，cost: 8.4367 seconds
Sliding window 4/709 is generated，cost: 8.4301 seconds
Sliding window 5/709 is generated，cost: 8.4504 seconds
Sliding window 6/709 is generated，cost: 8.4531 seconds


In [ ]:
G_dtw = dtw_graphs[0]  # 取出第一个滑动窗口的 DTW 图
print("Graph 1:")
node = 68  # 假设你想找节点 68 的边权重

# 遍历该节点的所有邻居，打印边的 DTW 权重
for neighbor, data in G_dtw[node].items():
    print(f"Edge {node} - {neighbor}, DTW Weight: {data['DTW_weight']}")

In [ ]:
node = 215801  # 假设你想找节点 68 的边权重

# 遍历该节点的所有邻居，打印边的 DTW 权重
for neighbor, data in G_dtw[node].items():
    print(f"Edge {node} - {neighbor}, DTW Weight: {data['DTW_weight']}")

In [ ]:
len(G_dtw.edges)

In [ ]:
merged_graphs_first.edges

In [ ]:
edge_index = torch.tensor(list(G_dtw.edges()), dtype=torch.long).t().contiguous()
edge_weight = torch.tensor([G_dtw[u][v]['DTW_weight'] for u, v in G_dtw.edges()], dtype=torch.float)

In [ ]:
print("edge_index [:, :20]:", edge_index[:, :50])
print("edge_weight [:, :20]:", edge_weight[:50])

In [ ]:
print("edge_index [:, -20:]:", edge_index[:, -50:])  # 取最后 20 条边
print("edge_weight [:, -20:]:", edge_weight[-50:])  # 取最后 20 个权重